# Dynamic Connectivity Python Pipeline

Given a subject number, this pipeline seeks to: 

1) Extract fMRI BOLD scan(s) into Atlas

2) Bin ROI Time-series into Predefined Time-Windows

3) Compute Connectomes from each Time-Window

4) Binarize-Threshold the Resulting Matrices

5) Concatenate the Time-Window Matrices into a 3D Array

6) Output 3D Matrices into MATLAB readable format

In [36]:
import os
import glob
import nilearn
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
from nilearn import plotting
from nilearn import input_data
import nibabel as nib
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import scipy.io
from nilearn import datasets



In [37]:
"""FUNCTIONS"""


def extract_fMRI_into_atlas(sub, path, pattern, scan_num, confounds_directory, atlas_dict, output_label, output_directory):
    # Find all relevant files that match filename "pattern" for subject number "sub". If number of files exceeds "scan_num",
    # select the "scan_num" largest files. Creates a numpy 2D array of shape (number of TRs, number of ROIs)
    directory_path = "%s/sub-%s/func" % (path, sub)
    scan_names = glob.glob(directory_path + "/*Ex*preproc_bold.nii.gz")
    
    if len(scan_names) > scan_num:
        scan_names.sort(key=lambda f: os.stat(f).st_size, reverse=True)
    
    # Check if multiple atlases are provided, if so, extract ROIs and concatenate

    for scan in scan_names:

        # Get corresonding confounds .tsv files for ith scan
        base = "_".join(scan.split("/")[-1].split("_")[:3])
        confounds_filename = confounds_directory + base + '_conformatted.tsv'
        pandas.read_csv(confounds_filename, sep = '\t').fillna(0).to_csv("conf_nona_temp.tsv", sep = '\t', index = False)

        print(scan)
        print("---" + confounds_filename)

        # Load fmri volumes
        func_img = nib.load(scan)
        header_zooms = func_img.header.get_zooms()
        TR = header_zooms[3]
        
        # Initialize Time Series
        time_series = []
        
        for atlas_filename, masker in atlas_dict.items():
            # Check which masker to use, then create a masker object and extract scan data into ROI timeseries using masker
            if masker == 'Maps':
                masker = input_data.NiftiMapsMasker(
                    atlas_filename, 
                    resampling_target='data',
                    memory = 'nilearn_cache', 
                    verbose = 0, low_pass=0.1, high_pass=0.01, detrend=True, t_r = TR).fit()
                time_series.append(masker.transform(scan, confounds = 'conf_nona_temp.tsv'))
            elif masker == "Labels":
                masker = NiftiLabelsMasker(
                    labels_img=atlas_filename,
                    standardize=True,
                    memory='nilearn_cache', 
                    verbose=0, low_pass = 0.1, high_pass = 0.01, detrend = True, t_r = TR)
                time_series.append(masker.fit_transform(scan, confounds = 'conf_nona_temp.tsv'))

        # Save timeseries to output directory, using output label and base name for reference
        time_series = np.concatenate(time_series, axis = 1)
        np.save("%s/%s_%s_timeseries.npy" % (output_directory, base, output_label), time_series)


def bin_time_series(sub, path, pattern, window_step, window_size):
    # Get scan filenames from subject id and pattern. Initialize empty containers for storing 
    # the resulting binned time windows and the corresponding start-end indices
    scan_filenames = glob.glob(path + "/sub-%s" % sub + pattern)
    scan_time_windows = {}
    indices = []
    
    # For each time series, create a range of start-end indices based on window step and size parameters. 
    # Bin the time series by the indices, and store the resulting sliding time windows into dicitonary 
    # where each key is a separate scan and its corresponding item is its 3D array of sliding time windows  
    # of shape (time window, TR number, ROI)
    for ts in scan_filenames:

        time_series = np.load(ts)

        time_windows = []
        for i in range(0, time_series.shape[0]-window_size + 1, window_step):
            window = time_series[i:i + window_size, :]
            time_windows.append(window)
            indices.append([i, i+window_size])

        
        scan_time_windows[ts] = np.asarray(time_windows)
    print(indices)
    return(scan_time_windows)


def connectomes_from_time_windows(time_windows_dict, kind):
    # Initialize a connectivity, of type "kind", as well as a dictionary to store the connectomes
    connectivity = ConnectivityMeasure(kind = kind)
    connectomes_dict = {}
    
    # For each scan's 3D array of time windows, compute their connectomes
    for timeseries_name, time_windows in time_windows_dict.items():
        connectomes_dict[timeseries_name] = connectivity.fit_transform(time_windows)
    
    return(connectomes_dict)


def binarize_threshold(connectomes_dict, threshold):
    # For each 3D Array of Connectomes, go through each Time Window and Binarize it according to 
    # some quantile threshold. Fill diagonal with 0. Return dictionary of 3D Connectome / Scan
    binthresh_connectomes_dict = connectomes_dict.copy()
    
    for timeseries_name, connectomes in binthresh_connectomes_dict.items():
        
        for i in range(connectomes.shape[0]):
            connectome = connectomes[i].copy()
            np.fill_diagonal(connectome, 0)
            abs_thresh = np.quantile(np.abs(connectome), q = threshold)
            connectome[(connectome>=-abs_thresh) & (connectome<=abs_thresh)] = 0
            connectome[connectome != 0] = 1
            connectomes[i] = connectome

        binthresh_connectomes_dict[timeseries_name] = connectomes
        
    return binthresh_connectomes_dict


def save_3D_array(binthresh_connectomes_dict, pattern, output_directory, output_label):
    scipy.io.savemat('%s/%s_%s_3Darray.mat' % (output_directory, pattern, output_label), binthresh_connectomes_dict)
        

In [47]:
%%timeit

sub = "033"
path = "/mnt/chrastil/lab/data/MLINDIV/preprocessed/derivatives/fmriprep"
confounds_directory = "/mnt/chrastil/lab/users/rob/projects/MachineLearning/confounds/"
timeseries_directory = "/mnt/chrastil/lab/users/rob/projects/DynConn/timeseries"
array_directory = "/mnt/chrastil/lab/users/rob/projects/DynConn/3DArrays"
atlas_filename = '/mnt/chrastil/lab/users/rob/projects/MachineLearning/MLINDIV-DL_251_80_rest.nii.gz'
masker = "Maps"
pattern = "Ex"
ts_pattern = "*Ex*MSDL*"
scan_num = 2
output_label = "MSDL-80"
threshold = 0.9


# HO + Schaeffer Atases
output_label = "HOsubcort+Schaefer400_2mm"
ts_pattern = "*Ex*HOsubcort+Schaefer400_2mm*"

schaefer = datasets.fetch_atlas_schaefer_2018(resolution_mm=2, verbose = 0)
schaefer_atlas = schaefer.maps
schaefer_labels = schaefer.labels
schaefer_masker = "Labels"

ho = datasets.fetch_atlas_harvard_oxford("sub-maxprob-thr25-2mm", verbose = 0)
ho_atlas = ho.maps
ho_labels = ho.labels
ho_masker = "Labels"

atlas_dict = {
    schaefer_atlas: schaefer_masker,
    ho_atlas: ho_masker
}


# Code to xecute w/ above parameters
extract_fMRI_into_atlas(sub, path, pattern, scan_num, confounds_directory, atlas_dict, output_label, timeseries_directory)
time_windows = bin_time_series(sub, timeseries_directory, ts_pattern, window_step = 15, window_size = 45)
connectomes = connectomes_from_time_windows(time_windows, kind = 'correlation')
binthresh_connectomes = binarize_threshold(connectomes, threshold)
save_3D_array(binthresh_connectomes, pattern, array_directory, output_label)

/mnt/chrastil/lab/data/MLINDIV/preprocessed/derivatives/fmriprep/sub-033/func/sub-033_task-boldEx_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
---/mnt/chrastil/lab/users/rob/projects/MachineLearning/confounds/sub-033_task-boldEx_run-1_conformatted.tsv
/mnt/chrastil/lab/data/MLINDIV/preprocessed/derivatives/fmriprep/sub-033/func/sub-033_task-boldEx2_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
---/mnt/chrastil/lab/users/rob/projects/MachineLearning/confounds/sub-033_task-boldEx2_run-1_conformatted.tsv
[[0, 45], [15, 60], [30, 75], [45, 90], [60, 105], [75, 120], [90, 135], [105, 150], [120, 165], [135, 180], [150, 195], [165, 210], [180, 225], [195, 240], [210, 255], [225, 270], [240, 285], [255, 300], [270, 315], [285, 330], [300, 345], [315, 360], [330, 375], [345, 390], [360, 405], [375, 420], [390, 435], [405, 450], [420, 465], [435, 480], [450, 495], [465, 510], [480, 525], [495, 540], [510, 555], [525, 570], [540, 585], [555, 600], [570, 615], [585, 630], [

In [45]:
time_windows

{}